# Imports

In [36]:
import requests
import ast
import pendulum
import numpy as np

# 1. Get list of all possible words

In [37]:
def get_remote_file(path):
    text = requests.get(path)
    return text.text

In [10]:
full_file = get_remote_file("https://www.nytimes.com/games/wordle/main.e17c80f8.js")

In [11]:
word_list_str = full_file[full_file.index("var mo=")+7:full_file.index("fanny\"]")+7]
word_list = ast.literal_eval(word_list_str)

In [12]:
print(len(word_list))

2309


In [13]:
word_list[:10]

['cigar',
 'rebut',
 'sissy',
 'humph',
 'awake',
 'blush',
 'focal',
 'evade',
 'naval',
 'serve']

# 2. Easy Solution

The word list is chronological, it contains all the words in the order they are going to appear on wordle. So it is possible to create a dictionary for each word with the date it will appear on wordle.

In [14]:
todays_date = "2022-04-25"
todays_word = "askew"

In [15]:
words_dict = {}
words_dict[todays_date] = todays_word

In [16]:
# get all words in word_list after todays_word:
word_list_future = word_list[word_list.index(todays_word)+1:]

for word in word_list_future:
    words_dict[pendulum.parse(str(todays_date)[0:10]).add(days=1).format("YYYY-MM-DD")] = word
    todays_date = pendulum.parse(str(todays_date)[0:10]).add(days=1).format("YYYY-MM-DD")

todays_date = pendulum.parse("2022-04-25")

# get all words in word_list before todays_word:
word_list_past = word_list[:word_list.index(todays_word)]

for word in word_list_past[::-1]:
    words_dict[pendulum.parse(str(todays_date)[0:10]).subtract(days=1).format("YYYY-MM-DD")] = word
    todays_date = pendulum.parse(str(todays_date)[0:10]).subtract(days=1).format("YYYY-MM-DD")

In [17]:
print(dict(list(words_dict.items())[-10:]))

{'2021-06-28': 'serve', '2021-06-27': 'naval', '2021-06-26': 'evade', '2021-06-25': 'focal', '2021-06-24': 'blush', '2021-06-23': 'awake', '2021-06-22': 'humph', '2021-06-21': 'sissy', '2021-06-20': 'rebut', '2021-06-19': 'cigar'}


Now we will sort the dictionary by order of words based on word_list. This step is not necessary but knowing that the dictionary is sorted chronologically helps me sleep better at night.

In [18]:
sorted_words_dict = {}
for word in word_list:
    word_date = list(words_dict.keys())[list(words_dict.values()).index(word)]
    sorted_words_dict[word_date] = word

In [19]:
# Printing last 10 wordles!
print(dict(list(sorted_words_dict.items())[-10:]))

{'2027-10-05': 'hydro', '2027-10-06': 'liege', '2027-10-07': 'octal', '2027-10-08': 'ombre', '2027-10-09': 'payer', '2027-10-10': 'sooth', '2027-10-11': 'unset', '2027-10-12': 'unlit', '2027-10-13': 'vomit', '2027-10-14': 'fanny'}


Getting a days wordle is as easy as accessing the index in sorted_words_dict by the date!

In [48]:
def get_todays_wordle():
    return sorted_words_dict[pendulum.parse(str(pendulum.today())[0:10]).format("YYYY-MM-DD")]

print(get_todays_wordle())

shown


Lets also write a function to get the wordle for any date

In [49]:
def get_wordle_date(year, month, day):
    if len(str(month))==1:
        month = "0"+str(month)
    if len(str(day))==1:
        day = "0"+str(day)
    try:
        return sorted_words_dict[pendulum.parse(str(year)+"-"+str(month)+"-"+str(day)).format("YYYY-MM-DD")]
    except:
        return "No wordle for that date!"

print(get_wordle_date(year=2022, month=4, day=28))

zesty


# 3. Proper Solution

We will solve Wordle using an algorithm which takes in feedback like a human and makes smart guesses!

## FAILED ATTEMPT! :

In [22]:
# FAILED ATTEMPT!

class WordleSolver():
    def __init__(self, possible_words):
        self.possible_words = possible_words
        pass

    def letter_values(self, result, guess):
        """Finds values of each letter"""
        result_outcome = []
        for i in range(0, 5):
            if result[i] == "b":
                result_outcome.append([guess[i], "b"])
            elif result[i] == "y":
                result_outcome.append([guess[i], "y"])
            elif result[i] == "g":
                result_outcome.append([guess[i], "g"])
        return result_outcome

    
    def word_remover(self, result, guess):
        """Returns the list of words with incorrect possibilties removed"""
        usable_words = []
        c = 0
        for item in self.letter_values(result, guess):
            if item[1] == "b":
                for word in self.possible_words:
                    if item[0] not in word:
                        usable_words.append(word)
            elif item[1] == "y":
                for word in self.possible_words:
                    if item[0] not in word[c]:
                        usable_words.append(word)
            elif item[1] == "g":
                for word in self.possible_words:
                    if item[0] in word[c]:
                        usable_words.append(word) 
            c+=1
        
        return usable_words
    

    def letterFreq(self):
        """Finds frequencies of letters in each position"""
        alphabet = "abcdefghijklmnopqrstuvwxyz"
        arr = {}
        for c in alphabet:
            freq = [0, 0, 0, 0, 0]
            for i in range(0, 5):
                for w in self.possible_words:
                    if w[i] == c:
                        freq[i] += 1
            arr.update({c: freq})
        return arr
    
    def wordScore(self, frequencies):
        """Computes a score based off letter frequencies"""
        words = {}
        max_freq = [0, 0, 0, 0, 0]
        for c in frequencies:
            for i in range(0, 5):
                if max_freq[i] < frequencies[c][i]:
                    max_freq[i] = frequencies[c][i]
        for w in self.possible_words:
            score = 1
            for i in range(0, 5):
                c = w[i]
                score *= 1 + (frequencies[c][i] - max_freq[i]) ** 2
            words.update({w: score})
        return words
    
    def bestWord(self, frequencies):
        """Finds the best word"""
        max_score = 1000000000000000000     # start with a ridiculous score
        best_word = "words"     # start with a random word
        scores = self.wordScore(frequencies)
        for w in self.possible_words:
            if scores[w] < max_score:
                max_score = scores[w]
                best_word = w
        return best_word
    
    def wordleSolver(self):
        """Prompts you to solve Wordle"""
        print("Welcome to the Wordle Solver!")
        print("The suggested starting word is:", self.bestWord(self.letterFreq()))
        guess = self.bestWord(self.letterFreq())
        print("Enter your first result:")
        result = input()
        counter = 1
        while result != "ggggg" and counter < 6:
            self.possible_words = self.word_remover(result, guess)
            print(self.possible_words)
            if len(self.possible_words) == 0:
                break
            suggestion = self.bestWord(self.letterFreq())
            print("The suggested word is:", suggestion)
            guess = suggestion
            print("Enter your new result:")
            result = input()
            counter += 1
        if counter == 6 and result != "ggggg":
            print("Number of guesses exceeded, sorry we failed!")
        else:
            print("Congratulations! We solved today's Wordle in", counter, "guesses.")

## SUCCESSFUL ATTEMPT! :    

In [44]:
# FINALLY WORKS!

def badLetters(result, guess):
    """Finds incorrect letters in word"""
    bad_letters = []
    for i in range(0, 5):
        if result[i] == "b":
            bad_letters.append(guess[i])
    return bad_letters

def partialLetters(result, guess):
    """Finds correct letters that are misplaced in word"""
    partial_letters = []
    for i in range(0, 5):
        if result[i] == "y":
            partial_letters.append([guess[i], i])
    return partial_letters

def correctLetters(result, guess):
    """Finds fully correct letters in word"""
    correct_letters = []
    for i in range(0, 5):
        if result[i] == "g":
            correct_letters.append([guess[i], i])
    return correct_letters

def word_remover(result, guess, possible_words):
    """Returns the list of words with incorrect possibilties removed"""
    bad_letters = badLetters(result, guess)
    correct_letters = correctLetters(result, guess)
    partial_letters = partialLetters(result, guess)
    good_letters = []
    for g in correct_letters:
        good_letters.append(g[0])
    for p in partial_letters:
        good_letters.append(p[0])
    
    acceptable_words1 = []
    for w in possible_words:
        check = 0
        for b in bad_letters:
            if b in w:
                if b in good_letters:
                    pass
                else:
                    check = 1
                    break
        if check == 0:
            acceptable_words1.append(w)

    acceptable_words2 = []
    for w in acceptable_words1:
        check = 0
        for g in correct_letters:
            if w[g[1]] != g[0]:
                check = 1
                break
        if check == 0:
            acceptable_words2.append(w)
    
    acceptable_words3 = []
    for w in acceptable_words2:
        check = 0
        for p in partial_letters:
            if w[p[1]] == p[0]:
                check = 1
                break
        if check == 0:
            acceptable_words3.append(w)
    
    acceptable_words4 = []
    for w in acceptable_words3:
        check = 0
        for g in good_letters:
            if g not in w:
                check = 1
                break
        if check == 0:
            acceptable_words4.append(w)

    acceptable_words5 = []
    for w in acceptable_words4:
        check = 0
        for b in bad_letters:
            if b in good_letters:
                if w.count(b) != good_letters.count(b):
                    check = 1
                    break
        if check == 0:
            acceptable_words5.append(w)
    
    return acceptable_words5

def letterFreq(possible_words):
    """Finds frequencies of letters in each position"""
    alphabet = "abcdefghijklmnopqrstuvwxyz"
    arr = {}
    for c in alphabet:
        freq = [0, 0, 0, 0, 0]
        for i in range(0, 5):
            for w in possible_words:
                if w[i] == c:
                    freq[i] += 1
        arr.update({c: freq})
    return arr

def wordScore(possible_words, frequencies):
    """Computes a score based off letter frequencies"""
    words = {}
    max_freq = [0, 0, 0, 0, 0]
    for c in frequencies:
        for i in range(0, 5):
            if max_freq[i] < frequencies[c][i]:
                max_freq[i] = frequencies[c][i]
    for w in possible_words:
        score = 1
        for i in range(0, 5):
            c = w[i]
            score *= 1 + (frequencies[c][i] - max_freq[i]) ** 2
        words.update({w: score})
    return words

def bestWord(possible_words, frequencies):
    """Finds the best word"""
    max_score = 1000000000000000000     # start with a ridiculous score
    best_word = "words"     # start with a random word
    scores = wordScore(possible_words, frequencies)
    for w in possible_words:
        if scores[w] < max_score:
            max_score = scores[w]
            best_word = w
    return best_word

def wordleSolver(possible_words):
    """Prompts you to solve Wordle"""
    print("Welcome to the Wordle Solver!")
    first_suggestion = bestWord(possible_words, letterFreq(possible_words))
    print("The suggested starting word is:", first_suggestion)
    guess = first_suggestion
    print("Enter your first result:")
    result = input()
    counter = 1
    while result != "ggggg" and counter < 6:
        possible_words = word_remover(result, guess, possible_words)
        print(possible_words)
        if len(possible_words) == 0:
            break
        suggestion = bestWord(possible_words, letterFreq(possible_words))
        print("The suggested word is:", suggestion)
        guess = suggestion
        print("Enter your new result:")
        result = input()
        counter += 1
    if len(possible_words) == 0:
        print("Oh no! You made a mistake entering one of your results. Please try again.")
    elif counter == 6 and result != "ggggg":
        print("Number of guesses exceeded, sorry we failed!")
    else:
        print("Congratulations! We solved today's Wordle in", counter, "guesses.")

In [47]:
wordleSolver(word_list)

Welcome to the Wordle Solver!
The suggested starting word is: slate
Enter your first result:
['sissy', 'shrub', 'spicy', 'sonic', 'shown', 'scour', 'showy', 'shrug', 'soggy', 'scorn', 'shock', 'skimp', 'sunny', 'sorry', 'snuck', 'spiny', 'sushi', 'scion', 'shook', 'sound', 'sworn', 'scrub', 'swoon', 'snuff', 'snoop', 'suing', 'shirk', 'spurn', 'swish', 'shuck', 'skunk', 'smock', 'synod', 'syrup', 'swung', 'spunk', 'smoky', 'shush', 'smirk', 'shiny', 'sword', 'spoof', 'squib', 'spoon', 'skiff', 'scoop', 'swing', 'scoff', 'swoop', 'scrum', 'sprig', 'sniff', 'spiky', 'spook', 'shorn', 'snowy']
The suggested word is: shook
Enter your new result:
['shown', 'showy', 'shorn']
The suggested word is: shown
Enter your new result:
Congratulations! We solved today's Wordle in 3 guesses.


**Works Great!**

# 4.Testing Average Guesses

Since we now have an algorithm that can solve wordle, lets try to see how mnay guesses it requires on average to get the word right. We will try all the word in our words list and test the bot against them. 

We will need to make small changes to our bot to enable the automatic testing:

In [26]:
def get_tries(possible_words, word_to_guess):
    """Prompts you to solve Wordle"""
    first_suggestion = bestWord(possible_words, letterFreq(possible_words))
    guess = first_suggestion
    print(guess)
    def get_letter_colors(actual, guess):
        # There are three possible conditions for every letter in the guess word:
        # Correct letter, correct position (green)
        # Correct letter, incorrect position (orange)
        # Incorrect letter (red)
        letterColors = []
        
        actualLetterCounts = {}
        for letter in actual:
            if letter not in actualLetterCounts:
                actualLetterCounts[letter] = 1
            else:
                actualLetterCounts[letter] += 1

        guessLetterCounts = {}        
        for letter in guess:
            if letter not in guessLetterCounts:
                guessLetterCounts[letter] = 1
            else:
                guessLetterCounts[letter] += 1

        for letterIndex in range(0, 5):
            if guess[letterIndex] == actual[letterIndex]:
                letterColors.append((guess[letterIndex], 'g'))
        # Only this one condition can lead to the duplicate problem.
        # So, instead of just checking whether the letter is in the word,
        # we have to check how many of it are in the guess versus how many
        # of it are in the actual word.  If it's in the wrong place and the
        # count is higher in guess than actual, it is a wrong letter:
        # color it red.  Otherwise, it's correct but misplaced:  color it orange.
            elif guess[letterIndex] in actual:
                if guessLetterCounts[guess[letterIndex]] > actualLetterCounts[guess[letterIndex]]:
                    letterColors.append((guess[letterIndex], 'b'))
                    # Decrement the count once the superfluous letter has been labeled incorrect
                    guessLetterCounts[guess[letterIndex]] -= 1
                else:
                    letterColors.append((guess[letterIndex], 'y'))
            else:
                letterColors.append((guess[letterIndex], 'b'))
        return letterColors

    result = get_letter_colors(word_to_guess, guess)
    result_str = ""
    for x in result:
        result_str += x[1]
    result = result_str

    counter = 1
    while result != "ggggg" and counter < 6:
        possible_words = word_remover(result, guess, possible_words)
        print(possible_words)
        if len(possible_words) == 0:
            break
        suggestion = bestWord(possible_words, letterFreq(possible_words))
        guess = suggestion
        result = get_letter_colors(word_to_guess, guess)
        result_str = ""
        for x in result:
            result_str += x[1]
        result = result_str


        counter += 1
    if counter == 6 and result != "ggggg":
        return -1
    else:
        return counter

In [27]:
tries = []
for word in word_list:
    tries.append(get_tries(word_list, word))

slate
['cigar', 'karma', 'major', 'marry', 'croak', 'maxim', 'parry', 'gamma', 'briar', 'gaudy', 'vodka', 'radio', 'favor', 'panic', 'aroma', 'comma', 'foray', 'cargo', 'hairy', 'canny', 'bayou', 'manor', 'apron', 'cacao', 'gawky', 'madam', 'augur', 'wacky', 'angry', 'aphid', 'dandy', 'carry', 'rainy', 'daddy', 'aging', 'mocha', 'pagan', 'human', 'manga', 'groan', 'fairy', 'ardor', 'nanny', 'handy', 'ninja', 'vapor', 'acorn', 'aping', 'mania', 'borax', 'china', 'rabid', 'caddy', 'magic', 'macaw', 'candy', 'march', 'arbor', 'hardy', 'razor', 'wagon', 'cabin', 'fancy', 'friar', 'dairy', 'among', 'rabbi', 'agony', 'urban', 'rajah', 'gonad', 'adorn', 'mafia', 'array', 'vapid', 'acrid', 'mayor', 'randy', 'radar', 'canon', 'pizza', 'baggy', 'mammy', 'nadir', 'havoc', 'annoy', 'kappa', 'bawdy', 'radii', 'dogma', 'nomad', 'cocoa', 'mango', 'rumba', 'audio', 'manic', 'admin', 'macho', 'harpy', 'cairn', 'broad', 'axiom', 'armor', 'jazzy', 'arrow', 'vicar', 'cobra', 'macro', 'baron', 'bacon', 'um

In [28]:
# Get number of unsuccessful tries:
tries.count(-1)

14

In [35]:
# % Successful
print(f"{((len(tries)-tries.count(-1)) / len(tries)) * 100:.2f}%")

99.39%


In [33]:
# % Unsuccessful:
print(f"{(tries.count(-1) / len(tries)) * 100:.2f}%")

0.61%


In [30]:
# Remove all -1's from tries
tries_successful = [i for i in tries if i != -1]

Lets see how many turns we need on average to get the word right:

In [31]:
np.mean(tries_successful)

3.6549019607843136

We need around 3.65 turns to get the word right! Not bad.

# Done! (I've wasted too much time on this 😅)